In [36]:
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
import numpy as np
from sklearn.cluster import MeanShift
from sklearn import preprocessing,cross_validation
import pandas as pd

df = pd.read_excel("titanic.xls")
original_df = pd.DataFrame.copy(df) # Feed through meanshift first and further analyse
print(df.head())




   pclass  survived                                             name     sex  \
0       1         1                    Allen, Miss. Elisabeth Walton  female   
1       1         1                   Allison, Master. Hudson Trevor    male   
2       1         0                     Allison, Miss. Helen Loraine  female   
3       1         0             Allison, Mr. Hudson Joshua Creighton    male   
4       1         0  Allison, Mrs. Hudson J C (Bessie Waldo Daniels)  female   

       age  sibsp  parch  ticket      fare    cabin embarked boat  body  \
0  29.0000      0      0   24160  211.3375       B5        S    2   NaN   
1   0.9167      1      2  113781  151.5500  C22 C26        S   11   NaN   
2   2.0000      1      2  113781  151.5500  C22 C26        S  NaN   NaN   
3  30.0000      1      2  113781  151.5500  C22 C26        S  NaN   135   
4  25.0000      1      2  113781  151.5500  C22 C26        S  NaN   NaN   

                         home.dest  
0                     St Louis,

In [37]:
# convert first into a numerical data -: sex ----> numerical data 

df.drop(['body','name'],1,inplace=True)
df.head()
df.fillna(0,inplace=True) #useful for filling empty cell in the data 

In [38]:
df.fillna(0,inplace=True) #useful for filling empty cell in the data 
df.head()

,pclass,survived,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,home.dest
0,1,1,female,29.0000,0,0,24160,211.3375,B5,S,2,"St Louis, MO"
1,1,1,male,0.9167,1,2,113781,151.5500,C22 C26,S,11,"Montreal, PQ / Chesterville, ON"
2,1,0,female,2.0000,1,2,113781,151.5500,C22 C26,S,0,"Montreal, PQ / Chesterville, ON"
3,1,0,male,30.0000,1,2,113781,151.5500,C22 C26,S,0,"Montreal, PQ / Chesterville, ON"
4,1,0,female,25.0000,1,2,113781,151.5500,C22 C26,S,0,"Montreal, PQ / Chesterville, ON"


In [43]:
#Hanling non numeric data 

def handle_non_numerical_data(df):
    columns = df.columns.values
    
    for column in columns:
        text_digit_vals = {}
        
        def convert_to_int(val):
            return text_digit_vals[val]
        
        if df[column].dtype != np.int64 and df[column].dtype != np.float64:
            column_contents = df[column].values.tolist()
            unique_elements = set(column_contents)
            x=0
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    text_digit_vals[unique] = x
                    x+=1
                    
            df[column] = list(map(convert_to_int,df[column]))
            
    return df

df = handle_non_numerical_data(df)
print(df.head())
                
df.drop(['ticket','home.dest'],1,inplace=True)



                    

   pclass  survived  sex      age  sibsp  parch      fare  cabin  embarked  \
0       1         1    1  29.0000      0      0  211.3375     79         0   
1       1         1    0   0.9167      1      2  151.5500     19         0   
2       1         0    1   2.0000      1      2  151.5500     19         0   
3       1         0    0  30.0000      1      2  151.5500     19         0   
4       1         0    1  25.0000      1      2  151.5500     19         0   

   boat  
0     1  
1    27  
2     0  
3     0  
4     0  


ValueError: labels ['ticket' 'home.dest'] not contained in axis

In [44]:
X = np.array(df.drop(['survived'],1).astype(float))
#scaling the data 
X = preprocessing.scale(X)
y = np.array(df['survived'])
print(y)

clf = MeanShift()
clf.fit(X)

[1 1 0 ..., 0 0 0]


/root/anaconda3/lib/python3.5/site-packages/sklearn/base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)


MeanShift(bandwidth=None, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     seeds=None)

In [45]:
labels = clf.labels_
cluster_centers = clf.cluster_centers_
n_clusters_ = len(np.unique(labels))

original_df['cluster_group'] = np.nan

for i in range(len(X)):
    original_df['cluster_group'].iloc[i] = labels[i]  # what happ-ens here is  iloc refers to the index of the row
    
survival_rates = {}

for i in range(n_clusters_):
    temp_df= original_df[ (original_df['cluster_group'] == float(i)) ] # original df cluster group is equal to i
                                                                         # we are creating a temporary dataframe only where cluster group label is i
    survival_cluster = temp_df[ (temp_df['survived']==1) ]
    survival_rate = len(survival_cluster)/len(temp_df)
    survival_rates[i] = survival_rate
    
print(survival_rates)

{0: 0.3761980830670926, 1: 1.0, 2: 0.6875, 3: 0.0, 4: 0.0, 5: 1.0}


/root/anaconda3/lib/python3.5/site-packages/pandas/core/indexing.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
